# FAQ recommendations

This Jupyter Notebook conducts an evaluation of various approaches to develop an LLM algorithm capable of recommending solutions based on an F.A.Q. database and the conversation context between an agent and a client.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1 handling the data

## 1.1.0 Converting F.A.Q from .txt to .csv file

In [4]:
def get_questions_and_answers(faq_path: str):
    questions = []
    answers = []

    with open(faq_path, "r") as file:
        for line in file.readlines():
            if "Pergunta" in line:
                question = line.lstrip("Pergunta: ").rstrip("\n")
                questions.append(question)
            elif "Resposta" in line:
                answer = line.lstrip("Resposta: ").rstrip("\n")
                answers.append(answer)
    
    return (questions, answers)


def create_csv(questions: list, answers: list, destination_path: str, file_name: str):
    faq_dict = {
        "Question": questions,
        "Answer": answers
        }
    
    path = destination_path + file_name + ".csv"

    df = pd.DataFrame(data=faq_dict, dtype=object)
    df.to_csv(path_or_buf=path, sep=";", index=False)

In [5]:
questions, answers = get_questions_and_answers(faq_path="../data/raw/FAQ_example.txt")
create_csv(questions=questions, answers=answers, destination_path="../data/raw/", file_name="FAQ_example")

# 2 Testing LLMs techniques

## 2.1 Sentence Embeddings pretrained models

In [95]:
from sentence_transformers import SentenceTransformer, util
import spacy
import re


def get_greatest_similarities(model, dataframe, message):
    questions_embeddings = model.encode(dataframe.Question)
    message_embed = model.encode(get_clean_text(message))

    similarities = util.cos_sim(message_embed, questions_embeddings)[0].tolist()
    greatest_similarities_index = pd.Series(similarities).sort_values(ascending=False).index
    
    return greatest_similarities_index


def get_clean_text(text:str):
    nlp_model = spacy.load("pt_core_news_sm")

    text_tokens = nlp_model(text)

    clean_text = []

    for token in text_tokens:
        if not token.is_stop and not token.is_punct:
            clean_text.append(token.lemma_)

    clean_text = " ".join(clean_text)

    return re.sub(r'[^a-zA-Z\s]', '', clean_text).lower()

In [96]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


In [97]:
message = "Boa noite, Queria como e saber como posso estar renovando meu plano"
message = get_clean_text(message)
print(message)

noite querer renovar plano


### 2.1.1 all-mpnet-base-v2 model

In [98]:
model = SentenceTransformer("all-mpnet-base-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

noite querer renovar plano


,Question,Answer
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
13,Como posso acompanhar meu consumo de internet ...,Você pode acompanhar seu consumo através do no...
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."


### 2.1.2 multi-qa-mpnet-base-dot-v1

In [99]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

noite querer renovar plano


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
14,A empresa oferece roteadores e equipamentos pa...,"Sim, fornecemos equipamentos para internet em ..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...


### 2.1.3 all-distilroberta-v1 model

In [100]:
model = SentenceTransformer("all-distilroberta-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

noite querer renovar plano


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."


### 2.1.4 all-MiniLM-L6-v2 model

In [101]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

noite querer renovar plano


,Question,Answer
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...


### 2.1.5 multi-qa-MiniLM-L6-cos-v1

In [102]:
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

noite querer renovar plano


,Question,Answer
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...


### 2.1.6 msmarco-distilbert-base-v4

In [103]:
model = SentenceTransformer("msmarco-distilbert-base-v4")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

noite querer renovar plano


,Question,Answer
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."


# 3 Preparing to Deploy

In [104]:
embeddings_model = SentenceTransformer("msmarco-distilbert-base-v4")
nlp_model = spacy.load("pt_core_news_sm")

In [105]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


## 3.1 Preparing Data

### 3.1.1 Creating Embeddings for each question and exporting as CSV file

In [106]:
def get_sentence_embeddings(model, series):
    """This function removes all 'stop words' for each sentence in a pandas.Series
    and then creates a embed for the sentence without the stop words."""

    text_embeddings = []

    for text in series:
        embed = model.encode(get_clean_text(text))
        text_embeddings.append(embed)

    return pd.Series(text_embeddings)

In [107]:
df_embeddings = get_sentence_embeddings(model, df.Question)
df_embeddings.head()

0    [0.16819185, 0.3711736, 0.11665192, 0.2539829,...
1    [0.52291733, 0.4691461, 0.023879822, 0.1911685...
2    [0.0087399455, -0.80580616, 1.0987761, 0.10045...
3    [-0.18636833, 0.061203647, 0.5096733, 0.349444...
4    [0.038230505, 0.17145498, -0.13606334, 0.31368...
dtype: object

In [108]:
df_embeddings.to_csv(path_or_buf="../data/processed/Embeddings.csv", sep=";", index=False)

## 3.2 Exporting the model with the best performance

### 3.2.1 Embeddings model

In [109]:
embeddings_model.save('../models/embed_model')

### 3.2.2 NLP Model

In [110]:
nlp_model.to_disk('../models/nlp_model')